In [1]:
#data load and inspection :
import pandas as pd
# Load CSV files
train_csv = pd.read_csv('Indonesian Food.v3i.multiclass_2/train/_classes.csv')
valid_csv = pd.read_csv('Indonesian Food.v3i.multiclass_2/valid/_classes.csv')
test_csv = pd.read_csv('Indonesian Food.v3i.multiclass_2/test/_classes.csv')

# Inspect the data
print(train_csv.head())
print(valid_csv.head())
print(test_csv.head())

                                            filename   ayam_bakar  \
0  240580_jpg.rf.5c33356651bdafa90c4f77466925b801...            0   
1  235615_jpg.rf.59536655137cb41d951de2ecb06f79ce...            0   
2  247361_jpg.rf.5bf64b4ff7cfad6fcbf4b6d498af01d6...            0   
3   7280_jpg.rf.5ab1f620eb50f736f61a5e744eee7ca9.jpg            0   
4  234091_jpg.rf.5b57ea07644880e794f708db3fe2fd1e...            0   

    ayam_goreng   batagor   bubur_ayam   mi_goreng   mie_ayam   nasi_goreng  \
0             1         0            0           0          0             0   
1             0         0            0           1          0             0   
2             0         0            1           0          0             0   
3             0         0            0           0          0             0   
4             0         0            0           1          0             0   

    sunny_side_egg  
0                0  
1                0  
2                0  
3                1  
4    

In [2]:
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

IMG_SIZE = (224, 224)
LABELS = ['ayam_bakar', 'ayam_goreng', 'batagor', 'bubur_ayam', 'mi_goreng', 'mie_ayam', 'nasi_goreng', 'sunny_side_egg']

def load_image_and_labels(csv_path, img_dir):
    df = pd.read_csv(csv_path)
    df.columns = df.columns.str.strip()  # Strip leading and trailing spaces from column names
    images = []
    labels = []
    
    for index, row in df.iterrows():
        img_path = os.path.join(img_dir, row['filename'])
        if not os.path.exists(img_path):
            continue  # Skip if the image file does not exist
        img = load_img(img_path, target_size=IMG_SIZE)
        img_array = img_to_array(img) / 255.0
        images.append(img_array)
        
        # Extract labels from the dataframe row
        label = [row[class_name] for class_name in LABELS]
        labels.append(label)
    
    return np.array(images), np.array(labels)

# Example usage
train_images, train_labels = load_image_and_labels('Indonesian Food.v3i.multiclass_2/train/_classes.csv', 'Indonesian Food.v3i.multiclass_2/train')
valid_images, valid_labels = load_image_and_labels('Indonesian Food.v3i.multiclass_2/valid/_classes.csv', 'Indonesian Food.v3i.multiclass_2/valid')
test_images, test_labels = load_image_and_labels('Indonesian Food.v3i.multiclass_2/test/_classes.csv', 'Indonesian Food.v3i.multiclass_2/test')


In [3]:
import tensorflow as tf

def preprocess_image(image, label):
    image = tf.image.resize(image, IMG_SIZE)
    return image, label

def augment_image(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.rot90(image, k=tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))
    image = tf.image.random_brightness(image, max_delta=0.25)
    return image, label

def create_tf_dataset(images, labels, batch_size=32, augment=False):
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    dataset = dataset.map(preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    if augment:
        dataset = dataset.map(augment_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.shuffle(buffer_size=1024).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

# Create datasets with augmentation for the training set
train_dataset = create_tf_dataset(train_images, train_labels, augment=True)
valid_dataset = create_tf_dataset(valid_images, valid_labels)
test_dataset = create_tf_dataset(test_images, test_labels)


In [4]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2

# Load the pre-trained MobileNetV2 model with input shape 224x224
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Unfreeze more layers of the base model for fine-tuning
for layer in base_model.layers[-40:]:
    layer.trainable = True

# Add custom layers on top of the base model with L2 regularization and Dropout
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu', kernel_regularizer=l2(0.001))(x)
x = Dropout(0.5)(x)
predictions = Dense(len(LABELS), activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model with a lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)

# Train the model
history = model.fit(train_dataset, validation_data=valid_dataset, epochs=50,
                    callbacks=[early_stopping, reduce_lr])


Epoch 1/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 102s 2s/step - accuracy: 0.3214 - loss: 3.0109 - val_accuracy: 0.6927 - val_loss: 2.0126 - learning_rate: 1.0000e-04
Epoch 2/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 78s 2s/step - accuracy: 0.6910 - loss: 1.9804 - val_accuracy: 0.7095 - val_loss: 1.9285 - learning_rate: 1.0000e-04
Epoch 3/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 79s 2s/step - accuracy: 0.8142 - loss: 1.7038 - val_accuracy: 0.7598 - val_loss: 1.7872 - learning_rate: 1.0000e-04
Epoch 4/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 78s 2s/step - accuracy: 0.8464 - loss: 1.5076 - val_accuracy: 0.7207 - val_loss: 1.9163 - learning_rate: 1.0000e-04
Epoch 5/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 77s 2s/step - accuracy: 0.9142 - loss: 1.3678 - val_accuracy: 0.7765 - val_loss: 1.7694 - learning_rate: 1.0000e-04
Epoch 6/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 77s 2s/step - accuracy: 0.9318 - loss: 1.3171 - val_accuracy: 0.7709 - val_loss: 1.7898 - learning_rate: 1.0000e-04
Epoch 7/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 80s 2s/step - accuracy: 0.9371 - loss

In [5]:
# Evaluate the model
val_loss, val_accuracy = model.evaluate(test_dataset)
print(f"Test Loss: {val_loss}")
print(f"Test Accuracy: {val_accuracy}")

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 375ms/step - accuracy: 0.8516 - loss: 1.5457
Test Loss: 1.495146632194519
Test Accuracy: 0.875


In [6]:
# Save the model in the recommended format
model.save('fitfood_finalModel.h5')

In [7]:
# Save the model in the recommended format
model.save('fitfood_finalModelk.keras')